In [ ]:
# !pip install thefuzz
# !pip install rapidfuzz

In [328]:
import pandas as pd
import glob
import rapidfuzz.utils
import thefuzz
from thefuzz import process, utils
from copy import copy
import ast
from rapidfuzz import fuzz as rapidfuzz


VALUE_ERROR = -1

##### Next cell is not relevant, you may skip it

In [10]:
path = 'data/job_links/processed/'
dfs = []
csv_files = glob.glob(f'{path}*.csv')
for chunk_id, file_name in enumerate(csv_files):
    df = pd.read_csv(file_name)
    df['chunk'] = [chunk_id] * df.shape[0]
    dfs.append(pd.read_csv(file_name))

In [194]:
path = 'data/profiles/raw_samples/'
dfs2 = []
csv_files = glob.glob(f'{path}*.csv')
for chunk_id, file_name in enumerate(csv_files):
    df = pd.read_csv(file_name)
    df['chunk'] = [chunk_id] * df.shape[0]
    dfs2.append(df)

In [195]:
merged_df_moti = pd.concat(dfs, axis=0)
merged_df_profiles = pd.concat(dfs2, axis=0)

In [196]:
choices = pd.read_csv('data/top_titles.csv').iloc[:, 0].to_list()

def get_match(title):
    matches = process.extractOne(title, choices, scorer=rapidfuzz.token_set_ratio,
                                 processor=thefuzz.process.default_processor)
    if matches[1] < 90:
        return None
    return matches[0]

In [197]:
df = merged_df_profiles
df = df[df.position.str.contains(r'[a-zA-Z]', na=False)].reset_index().rename(
    {'index': "position_idx"}, axis=1).drop_duplicates(subset=['position'], keep='first')
df['narrow_position'] = df['position'].apply(lambda x: get_match(x))
df.dropna(subset=['narrow_position'], inplace=True)

In [319]:
map_df = pd.read_csv('data/alternative_titles_map.csv').drop_duplicates(subset=['alternative_titles'],keep='first')
alt_df = pd.read_csv('data/alternate_titles.csv').rename({'Alternate Title':'alternative_titles','Title':'title'},axis=1)

def find_narrowed_match(pos_title,exp_title):
    try:
        title_item = map_df.loc[map_df['alternative_titles'] == pos_title, 'title'].item()
        find_match_in = ast.literal_eval(alt_df.loc[alt_df['title'] == title_item, 'alternative_titles'].values[0])
        matches = process.extractOne(exp_title, find_match_in, scorer=rapidfuzz.token_set_ratio, processor=thefuzz.process.default_processor)
        return matches[0]
    except Exception as e:
        return VALUE_ERROR

#### Adjust values in the next block:
> to_csv : weather to save the results
> 
> minimum_match_score: minimum required match score between position and prev_experience title for considering them "similar"
> 
> count_threshold: currently not relevant, threshold as described in the tasks google sheet section (2.c)

In [ ]:
to_csv = False
narrowed_size = len(choices)
minimum_match_score = 90
done_count = 0
count_threshold = 120

In [ ]:
records = {}
titles_count = {title: count_threshold for title in choices}
titles_collection_done = {title: False for title in choices}

for _, item in df.iterrows():
    record_item = copy(item.to_dict())
    orig_pos = item['position']
    pos = item['narrow_position']
    titles_count[pos] -= 1
    if not titles_collection_done[pos] and titles_count[pos] == 0:
        titles_collection_done[pos] = True
        done_count += 1
    experience = ast.literal_eval(item['title'])
    duration_short = ast.literal_eval(item['duration_in_years'])
    new_exp = []
    new_dur = []
    time_in_position = 0
    for i, exp in enumerate(experience):
        if rapidfuzz.partial_token_set_ratio(orig_pos, exp) >= minimum_match_score:
            time_in_position += duration_short[i]
        else:
            match_exp_title = find_narrowed_match(pos, experience[i])
            if match_exp_title != VALUE_ERROR:
                new_exp.append(match_exp_title)
                new_dur.append(duration_short[i])
    records[orig_pos] = record_item | {'years_in_position': time_in_position, 'prev_experience': new_exp,
                                       'experience_duration': new_dur}
    if done_count == narrowed_size:
        break
        
result_df = pd.DataFrame.from_dict(records, orient='index')
if to_csv:
    result_df.reset_index(drop=True).to_csv('data/processed_profiles.csv', index=False)